# Create and Run chat flows using Promptflow Python SDK

### Overview

Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality.

In this hands-on, you will be able to:
Create flows that link fine-tuned phi3.5 endpoint(Python code) and gpt model in a executable workflow.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.

#### 1. Set up Promptflow client with Credential and configuration

#### 2. Create a new chat flow by providing the flow name and description.

#### 3. Run Basic Promptflow with questions to compare models

#### 4. Run Context Added Promptflow with the outdoor questions

#### 5. Use serverless endpoint to run the Promptflow with context

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys

def get_lab_prep_dir(target_dir="slm-innovator-lab"):
    current_dir = os.getcwd()

    if target_dir in current_dir:
        lab_abs_dir = current_dir.split(target_dir)[0] + target_dir
    else:
        lab_abs_dir = current_dir 
        
    lab_prep_dir = os.path.join(lab_abs_dir, "0_lab_preparation")
    print("Lab prep dir: ", lab_prep_dir)    
    return lab_prep_dir

sys.path.append(os.path.abspath(get_lab_prep_dir()))

from common import check_kernel
check_kernel()

In [ ]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError
load_dotenv()
with open('./config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

In [ ]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Set up Promptflow client with Credential and configuration

-   Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name


In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# if you cannot use DefaultAzureCredential and InteractiveBrowserCredential you need to set up the Managed identity in your .env file

pf_azure_client = PFClient.from_config(credential=credential, path="./config.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


## Check the exist connections

-   currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.
    > ✨ **_important_** <br>
    > Update flow.dag.yaml files in your flow_path with the connection name you have created in the Azure ML Studio UI.


## 2. Create a new chat flow by providing the flow name and description.

-   Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI studio UI.


In [ ]:
#Storage Blob Data Contributor, Storage File Data Privileged Contributor role을 managed Identity / service principal / InteractiveBrowserCredential 에게 부여하면 아래 코드를 실행할 수 있습니다.
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")

## 2. Run Basic Promptflow with questions to compare models

-   Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models


In [ ]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_result)

In [ ]:
detail = pf_azure_client.get_details(run_result)

detail

## 3. Run Context Added Promptflow with the outdoor questions

-   Run the Promptflow using the context data and ask the outdoor product related questions to compare the results of the models


In [ ]:
flow_path = "./chat-context"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_context_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_result_with_context)

In [ ]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

## 4. Use serverless endpoint to run the Promptflow with context

-   Create a serverless endpoint to run the Promptflow with the context. You can use the endpoint to run the flow with the context.


### deploy your serverless endpoint

-   go to the Azure AI studio > Model catalog > search phi-3.5 > deply Phi-3.5-mini-instruct as your serverless endpint
    <br>
    ![serverless endpoint](./images/deploy_serverless_endpoint.jpg)
    <br>
    <br>
-   once the deployment is done, go to Deployments and you can see the endpoint deployed in the endpoint section. Click to check the details and copy key and phi35-mini-instruct: Chat Completion endpoint url
    ![copy connection](./images/copy_connection.jpg)
    <br>
    <br>
-   go to Settings in Azure AI studio > Connections > create a new connection naming phi35-serverless with the copied key and endpoint url
    ![create new serverless connection](./images/create_new_serverless_connection.jpg)


In [ ]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_serverless_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


In [ ]:
monitor_status(pf_azure_client, run_serverless_result)

In [ ]:
detail = pf_azure_client.get_details(run_serverless_result)

detail